In [1]:
from data_util import load_embeddings, DMConfig, tfConfig
try:
    import _pickle as cPickle
except ImportError:
    import cPickle
import gzip
import os
import numpy as np
from sklearn import metrics
from defs import LBLS

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
config = DMConfig()
if os.path.exists(config.tok2id_path):
    with gzip.open(os.path.abspath(config.tok2id_path)) as f:
        tok2id = cPickle.load(f)
if os.path.exists(config.id2tok_path):
    with gzip.open(os.path.abspath(config.id2tok_path)) as f:
        id2tok = cPickle.load(f)

## Training

In [21]:
from tf_random_classifier import TfRandomClassifier
from defs import PROJECT_DIR

In [22]:
model = TfRandomClassifier()
model.fit(tfConfig, max_iter=0)

INFO:Initializing data manager...
INFO:took 0 s
INFO:Building model...
INFO:took 0 s
INFO:-- Initializingt new model


## Evaluation

In [31]:
def print_example(pred, label, feats, length, email_id):
    ids_ = feats[:length][:, 0]
    print("==== {} ====".format(email_id))
    print("Predicted label: {}. True label: {}".format(LBLS[pred], LBLS[label]))
    print( " ".join((id2tok[i] for i in ids_)) )
    print("------------------------------------")


def print_examples(preds, outputs, inputs, lens, email_ids):
    for p, o, i, l, d in zip(preds, outputs, inputs, lens, email_ids):
        print_example(p, o, i, l, d)

In [33]:
outputs, preds, word_ids, email_ids = model.evaluate_tfdata(model.sess, dataset_name='test', batch_lim=100, writer=None)

preds = np.asarray(preds); outputs = np.array(outputs)    
correct = preds == outputs
prec, rec, f1, _ = metrics.precision_recall_fscore_support(outputs, preds, average='binary')

print(" Predicted {} of {} examples correctly".format(np.sum(correct), correct.shape[0]) )
print(" Precision: {:1.3f}, Recall: {:1.3f}, F1: {:1.3f}".format(prec, rec, f1) )

INFO:Ran 100 batches


 Predicted 1569 of 3200 examples correctly
 Precision: 0.333, Recall: 0.489, F1: 0.396


In [30]:
batch_preds, inputs, lens, batch_outputs, email_ids = model.predict()
print_examples(preds, outputs, inputs, lens, email_ids)

==== b'<30206745.1075845014402.JavaMail.evans@thyme>' ====
Predicted label: REPLY. True label: REPLY
<s> dear peggy : in UUUNKKK to our telephone conversation , please find below brief analysis of the UUUNKKK claim submitted by UUUNKKK UUUNKKK UUUNKKK UUUNKKK , s.a. de UUUNKKK ( `` UUUNKKK '' ) which you have the concern may affect the standing of permits granted to enron UUUNKKK UUUNKKK de mexico , s.a. de UUUNKKK ( `` enron '' ) . the main arguments presented by UUUNKKK in its UUUNKKK complaint seeks to challenge the UUUNKKK of certain legislation , including regulations under which permits are granted for generation of electricity . ? further , UUUNKKK challenges the actions of the energy regulatory ( commission UUUNKKK UUUNKKK de UUUNKKK ) UUUNKKK as it granted a permit to enron and other entities , despite the fact that UUUNKKK had submitted an application earlier in NNNUMMM . ? the challenge is also directed against action of the federal electricity commission ( UUUNKKK federal d